In [1]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import random
import time
import logging
import math
import sys
import os
import numpy as np
import pandas as pd
import copy
#import pygame
#from pygame import gfxdraw
from ipdb import set_trace as debug

In [2]:
A = np.array([[100,0],[0,200],[300,0],[0,-400]])
temp1 = A[0,:]
temp2 = A[2,:]

dist = np.linalg.norm(temp1-temp2)
dist

200.0

In [3]:
area_edge = [[-500,500],[-500,500]]
density = 4*10**(-6)

def ppp(area_edge, density):
        xMin = area_edge[0][0]
        xMax = area_edge[0][1]
        yMin = area_edge[1][0]
        yMax = area_edge[1][1]
        
        xDelta = xMax - xMin
        yDelta = yMax - yMin
        
        areaTotal = xDelta * yDelta
        
        node_num = np.random.poisson( density * areaTotal )
        node_x = xDelta * np.random.rand(node_num) + xMin
        node_y = yDelta * np.random.rand(node_num) + yMin
        return node_x, node_y

In [4]:
node_x, node_y = ppp(area_edge, density)

In [5]:
NUM_INTERF = len(node_y)
        
interf_pos = np.zeros((NUM_INTERF,2))
for j in range(NUM_INTERF):
    interf_pos[j] = [node_x[j], node_y[j]]

In [6]:
def distance(x,y):
        DIST = np.sqrt(np.sum((x-y)**2, axis=1))
        return DIST

In [7]:
NUM_SENSOR = 4
NUM_EDGE = 4
POS_SENSOR = np.array([[100,0],[0,200],[300,0],[0,-400]])
POS_EDGE = np.array([[-250,250],[250,250],[250,-250],[-250,-250]])
AREA_EDGE = np.array([[-500,500],[-500,500]])
USER_DENSITY = 4*10**(-6) # node/m^2
TX_POWER_DBM = 25
TX_POWER = 10 ** (TX_POWER_DBM/10)
NOISE_POWER_DBM = -80
NOISE_POWER = 10 ** (NOISE_POWER_DBM/10)
BANDWIDTH = 5 * (10 ** 6)
DATA_SIZE_MB = 2
DATA_SIZE = DATA_SIZE_MB * (8 * 10**6)

#### Main link power

In [8]:
main_DIST = distance(POS_SENSOR, POS_EDGE)
print(main_DIST)

[430.11626335 254.95097568 254.95097568 291.54759474]


In [9]:
main_channel = np.random.exponential(scale=1.0, size=(1,NUM_SENSOR))
print(main_channel)

[[0.007779   0.83344291 0.90952107 0.48079666]]


In [10]:
main_PL = main_DIST ** (-4)
print(main_PL)

[2.92184076e-11 2.36686391e-10 2.36686391e-10 1.38408304e-10]


In [11]:
main_link = TX_POWER * main_channel * main_PL
print(main_link)

[[7.18754506e-11 6.23805422e-08 6.80747495e-08 2.10437720e-08]]


#### Interference power

In [14]:
interf_power = np.zeros((1,NUM_SENSOR))

for i in range(NUM_SENSOR):
    interf_x, interf_y = ppp(AREA_EDGE, USER_DENSITY)
    NUM_INTERF = len(interf_y)
    
    POS_INTERF = np.zeros((NUM_INTERF,2))
    
    for j in range(NUM_INTERF):
        POS_INTERF[j] = [interf_x[j], interf_y[j]]
    
    interf_power_tmp = interference(NUM_INTERF, NUM_EDGE, POS_INTERF, POS_EDGE)
    interf_power_tmp_copy = copy.copy(interf_power_tmp)
    
    interf_power[0,i] = interf_power_tmp_copy[i]
    
    del interf_x, interf_y, NUM_INTERF, POS_INTERF, interf_power_tmp

print(interf_power)

[[2.52791128e-09 2.02260926e-09 2.67074242e-08 9.41992161e-07]]


In [15]:
interf_x, interf_y = ppp(AREA_EDGE, USER_DENSITY)
NUM_INTERF = len(interf_y)
print(interf_x)
print(interf_y)
print(NUM_INTERF)

[ 264.31620383 -303.46927548  201.45584949  497.65725591]
[ 377.59829703  178.34323133  414.80134127 -443.99003285]
4


In [16]:
POS_INTERF = np.zeros((NUM_INTERF,2))
for j in range(NUM_INTERF):
    POS_INTERF[j] = [interf_x[j], interf_y[j]]
print(POS_INTERF)
print(POS_INTERF[0])

[[ 264.31620383  377.59829703]
 [-303.46927548  178.34323133]
 [ 201.45584949  414.80134127]
 [ 497.65725591 -443.99003285]]
[264.31620383 377.59829703]


In [17]:
interf_DIST_1 = distance(POS_INTERF[0], POS_EDGE)
print(interf_DIST_1)

interf_DIST = distance(POS_INTERF, POS_EDGE[0])
print(interf_DIST)

interf_PL = interf_DIST ** (-4)
print(interf_PL)

interf_channel = np.random.exponential(scale=1.0, size=(1,NUM_INTERF))
print(interf_channel)

interf_links = TX_POWER * interf_channel * interf_PL
print(interf_links)

interf_power = sum(sum(interf_links))
print(interf_power)

[529.90799478 128.39890613 627.76155993 811.41899162]
[ 529.90799478   89.40724756  480.5953247  1020.10467013]
[1.26823026e-11 1.56498111e-08 1.87448441e-11 9.23466311e-13]
[[0.73782857 0.26715148 0.278339   3.36499951]]
[[2.95905871e-09 1.32210723e-06 1.64989346e-09 9.82666299e-10]]
1.327698852276604e-06


In [18]:
def interference(NUM_INTERF, NUM_EDGE, POS_INTERF, POS_EDGE):
    interf_power = np.zeros(NUM_EDGE)
    for count in range(NUM_EDGE):
        dist_to_edge = distance(POS_INTERF, POS_EDGE[count])
        interf_PL = dist_to_edge ** (-4)
        interf_channel = np.random.exponential(scale=1.0, size=(1,NUM_INTERF))
        interf_links = TX_POWER * interf_channel * interf_PL
        interf_link = sum(sum(interf_links))
        interf_power[count] = interf_link
    return interf_power

In [19]:
interfTMP=interference(NUM_INTERF, NUM_EDGE, POS_INTERF, POS_EDGE)
print(interfTMP)

[8.97865158e-07 8.14540198e-07 9.99604446e-08 1.75177164e-08]


In [20]:
for i in range(NUM_EDGE):
    
    print(interfTMP[i])
    

8.978651580617931e-07
8.145401983286774e-07
9.996044464799118e-08
1.75177164377361e-08


#### Calculate SINR

In [21]:
interf_link = interference(NUM_INTERF, NUM_EDGE, POS_INTERF, POS_EDGE)
SINR = np.round(main_link / (interf_link + NOISE_POWER),3)
print(SINR)

DATA_RATE = BANDWIDTH * np.log(1+SINR)
print(DATA_RATE)

[[0.    0.382 1.863 0.702]]
[[      0.         1617658.62672739 5259350.13057989 2659020.15075591]]


In [22]:
np.exp(0.4)

1.4918246976412703

In [23]:
DATA_RATE_CONST = 2 * (10**6)

In [24]:
DATA_RATE_compare = DATA_RATE > DATA_RATE_CONST
STP = DATA_RATE_compare*1
print(STP)

[[0 0 1 1]]


In [30]:
TX_TIME = np.round(DATA_SIZE / (DATA_RATE+10**(-6)), 3)
print(TX_TIME)

[[1.600e+13 9.891e+00 3.042e+00 6.017e+00]]


In [26]:
num_tx = np.sum((STP))
print(num_tx)

2


In [27]:
A=[[0,1,1,0]]
print(A)

[[0, 1, 1, 0]]


In [28]:
print(A[0])

[0, 1, 1, 0]
